# <center>Mini-Project: SVM & LR Classification</center>
<center><font size = "4"> 2017-2018 California Department of Education Mathmematics Achievement</font></center>

##### <center>Created by An Nguyen, Andy Ho, Jodi Pafford, Tori Wheelis</center>
<center> February 06, 2019</center>

In [369]:
import pandas as pd
import numpy as np
import datetime as dt
from __future__ import print_function
from sklearn.model_selection import ShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn import metrics as mt

rainfall_original = pd.read_csv('weatherAus.csv') 

In [370]:
rainfall = rainfall_original
rainfall.drop(["RISK_MM"], axis=1, inplace=True) #RISK_MM was used to extrapolate response variable.
rainfall['Date'] =  pd.to_datetime(rainfall['Date'], format='%Y-%m-%d') #change 'Date' variable to datetime64
rainfall.dropna(subset=["RainToday"], inplace=True) #drop any observation with no record of rainfall for the day, cannot be imputed
rainfall = rainfall.reset_index(drop=True) #reset the index after drops
rainfall.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140787 entries, 0 to 140786
Data columns (total 23 columns):
Date             140787 non-null datetime64[ns]
Location         140787 non-null object
MinTemp          140319 non-null float64
MaxTemp          140480 non-null float64
Rainfall         140787 non-null float64
Evaporation      81093 non-null float64
Sunshine         73982 non-null float64
WindGustDir      131624 non-null object
WindGustSpeed    131682 non-null float64
WindDir9am       131127 non-null object
WindDir3pm       137117 non-null object
WindSpeed9am     139732 non-null float64
WindSpeed3pm     138256 non-null float64
Humidity9am      139270 non-null float64
Humidity3pm      137286 non-null float64
Pressure9am      127044 non-null float64
Pressure3pm      127018 non-null float64
Cloud9am         88162 non-null float64
Cloud3pm         84693 non-null float64
Temp9am          140131 non-null float64
Temp3pm          138163 non-null float64
RainToday        140787 non-n

In [371]:
def impute_by_city(cities,variables):
    for c in cities:
        temp = rainfall[rainfall.Location == c]
        i = min(temp.index)
        while i < max(temp.index):
            for v in variables:
                if pd.isna(temp[v]).all():
                    pass
                elif pd.isna(temp[v][i]):
                    temp[v][i] = find_mean(temp[v], i)
                    rainfall[v][i] = temp[v][i]
            i
            i = i + 1       

def find_mean(templist, index):
    if index == min(templist.index):
        return find_top(templist, index)
    elif index == max(templist.index):
        return find_bottom(templist, index)
    else:
        bottom = find_bottom(templist, index)
        top = find_top(templist, index)
    mean = (top + bottom)/2
    return mean

def find_bottom(templist, index):
    while pd.isna(templist[index-1]):
        index = index-1
    bottom = templist[index-1]
    return bottom
    
def find_top(templist, index):
    while pd.isna(templist[index+1]):
        index = index+1
    top = templist[index+1]
    return top          

In [394]:
directions = {'N':0, 'NNE':22.5, 'NE':45, 'NE':45, 'ENE':67.5, 'E':90, 'ESE':112.5, 'SE':135, 'SSE':157.5, 'S':180,\
              'SSW':202.5, 'SW':225, 'WSW':247.5, 'W':270, 'WNW':292.5, 'NW':315, 'NNW':337.5}
cities = rainfall.Location.unique()
c_variables = []
d_variables = []
for l in list(rainfall):
    if (rainfall[l].dtypes == 'float64'):
        c_variables.append(l)
    else:
        d_variables.append(l)
        
rainfall = rainfall.replace(directions)
rainfall.RainToday = rainfall.RainToday=='Yes'
rainfall.RainToday = rainfall.RainToday.astype(np.int)
rainfall.RainTomorrow = rainfall.RainTomorrow=='Yes'
rainfall.RainTomorrow = rainfall.RainTomorrow.astype(np.int)

#impute_by_city(cities, c_variables)